In [1]:
import keras
import pandas as pd
import numpy
import random
import common_functions
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from matplotlib import pyplot

/glob/intel-python/versions/2018u2/intelpython3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
subformat = pd.read_csv("updated_submission_template_17112018.csv")
feq = pd.read_csv('del_freq.csv', index_col=0) #The csv file that has the data of delivery frequency
level_readings = pd.read_csv("processed_level.csv", header=0, index_col=0, squeeze=True)
level_readings.fillna(0, inplace=True)

/glob/intel-python/versions/2018u2/intelpython3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
#configuration variables
print_predictions = False
write_in_csv = True
option ={
    0:'BR-217970_freq0.h5',
    1:'freq1_US-7455.h5',
    2:'freq2_SP-121.h5',
    3:'freq3_UK-28848.h5',
    4:'freq4_UK-3689.h5',
    5:'freq5_CN-156811.h5',
    6:'freq6_TH-957.h5',
    7:'freq7_SP-31121.h5',
    8:'freq8_US-64768.h5',
    9:'freq9_CN-151511.h5',
    10:'freq10_IN-109046.h5',
    11:'freq11_UK-6441.h5',
    12:'freq12_TH-4391.h5',
    13:'freq13_IN-100687.h5',
    14:'freq14_TH-156112.h5',
    15:'freq14_TH-156112.h5',
    16:'freq16_US-219119.h5',
    17:'freq17_BR-218540.h5',
    18:'freq18_CN-154371.h5',
    19:'freq19_UK-6417.h5',
    20:'freq20_CN-152231.h5',
    'B':'CN-128211GRU40-sigmoid.h5' }


In [4]:
def predict(vessel_id):
           
    raw_values = level_readings[level_readings["VESSEL_ID"] == vessel_id]["INST_PRODUCT_AMOUNT"].values
    raw_values = raw_values.reshape(len(raw_values), 1)
    
    #Data Normalization:
    #diff_values replaces the level sensor reading with the difference of current timestamp with previous timestamp
    #Hence the machine learns only the difference not the exact value
    given_data = raw_values
    diff_values = common_functions.difference(raw_values)
    
    #Data Normalization:
    #Converting the values between 0 and 1
    scaler = MinMaxScaler(feature_range=(0, 1))    
    diff_values = numpy.expand_dims(diff_values, axis=1)
    scaler = scaler.fit(diff_values)
    dataset_scaled = scaler.transform(diff_values)
    
    #selecting the keras model based on the delivery frequency
    f = feq[feq['VESSEL_ID']== vessel_id ]['DELIVERY_FREQUENCY'].values
    if f[0] in option.keys():
        model_name = option[f[0]]
    else:
        model_name = option['B']
        
    lstm_model = keras.models.load_model("trained_models/" + model_name)
     
    
    #initiating the empty list to store the results
    predictions = list()
    flowrate_pred = list()
    flowrate_truth = list()
    for i in range(0,217):
        if i == 0:
            # begin with zero
            inp = 0
        else:
            # Use the previous timestamp value to get the next one
            inp = predictions[-1]

        X = numpy.array([[inp]])
        yhat = common_functions.forecast_lstm(lstm_model, 1, X)
        # invert scaling
        yhat = common_functions.invert_scale(scaler, X, yhat)
        
        # inverting difference
        # instead adding the value to the previously predicted value, it is added to 
        # value before 10 days which is available in the given data
        yhat = yhat + given_data[-217+i]
        
        #making sure the value doesn't go negative
        flowrate_pred.append((yhat - inp)[0] if (yhat - inp) < 0 else 0.0 )
        
        predictions.append(yhat)
        #expected = exp[i]
        #if print_predictions:
            #print(i , flowrate_pred[-1])
            # print(i , flowrate_pred[-1], flowrate_truth[-1])
            #print('Input = %d , Predicted=%f, Expected=%f' % (inp, yhat, expected))

    # report performance    
    #rmse = sqrt(mean_squared_error(flowrate_pred, flowrate_truth))
    #if write_in_csv:
    #print('updating the df for ', vessel_id)
    #write_indices = subformat_copy[subformat_copy['VESSEL_ID'] == vessel_id]['PREDICTION'].index.values
    #subformat_copy.loc[write_indices, "PREDICTION"] = flowrate_pred
    #print('Test RMSE: %.3f' % rmse)
    
    return flowrate_pred

In [5]:
a = predict('BR-216705')
print(a)

/glob/intel-python/versions/2018u2/intelpython3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
